In [15]:
import pickle
with open('train_sentence_transformer.pkl', 'rb') as f:
    train = pickle.load(f)
with open('val_sentence_transformer.pkl', 'rb') as f:
    val = pickle.load(f)
with open('test_sentence_transformer.pkl', 'rb') as f:
    test = pickle.load(f)

In [16]:
import json
with open('test.json', encoding = 'utf8') as json_file:
    data = json.load(json_file)
    
conversations = []
summaries = []
for i in range(0, len(data)):
    if len(data[i]['dialogue'].split('\r\n')) > 1:
        sentences = data[i]['dialogue'].split('\r\n')
    else:
        sentences = data[i]['dialogue'].split('\n')

    conversations.append(sentences)
    summaries.append(data[i]['summary'].strip('\n').replace('\r\nt', ' '))

In [17]:
import numpy as np
length = []
for i in range(0, len(train)):
    length.append(len(train[i]))
for i in range(0, len(val)):
    length.append(len(val[i]))
for i in range(0, len(test)):
    length.append(len(test[i]))
X = []
for i in range(0, len(train)):
    for j in range(0, len(train[i])):
        X.append(np.array(train[i][j]))
for i in range(0, len(val)):
    for j in range(0,len(val[i])):
        X.append(np.array(val[i][j]))
for i in range(0, len(test)):
    for j in range(0, len(test[i])):
        X.append(np.array(test[i][j]))

In [18]:
from hmmlearn import hmm

In [19]:
remodel = hmm.GaussianHMM(n_components=4, n_iter = 50, covariance_type = 'diag', verbose = True, init_params="cm", params="cmts")
remodel.startprob_ = np.array([1, 0.0, 0.0, 0.0])
remodel.transmat_ = np.array([
                 [0.33, 0.34, 0.33, 0],
                 [0.0, 0.33, 0.34, 0.33],
                 [0.0, 0.0, 0.5, 0.5],
                 [0.0, 0.0, 0.0, 1.0]]
                            )

In [20]:
remodel.startprob_

array([1., 0., 0., 0.])

In [21]:
remodel.transmat_

array([[0.33, 0.34, 0.33, 0.  ],
       [0.  , 0.33, 0.34, 0.33],
       [0.  , 0.  , 0.5 , 0.5 ],
       [0.  , 0.  , 0.  , 1.  ]])

In [22]:
remodel.fit(X, length)

         1 -277209.55747327             +nan
         2 -266144.68687608  +11064.87059720
         3 -265892.30516020    +252.38171587
         4 -265873.86530276     +18.43985744
         5 -265861.84329983     +12.02200293
         6 -265859.35037933      +2.49292050
         7 -265854.66361374      +4.68676560
         8 -265854.64934989      +0.01426385
         9 -265854.64934983      +0.00000006


GaussianHMM(init_params='cm', n_components=4, n_iter=50, params='cmts',
            verbose=True)

In [23]:
remodel.transmat_

array([[0.72868156, 0.15504703, 0.11627141, 0.        ],
       [0.        , 0.91537789, 0.02308566, 0.06153645],
       [0.        , 0.        , 0.90475957, 0.09524043],
       [0.        , 0.        , 0.        , 1.        ]])

In [24]:
remodel.startprob_

array([1., 0., 0., 0.])

In [25]:
def encode_convs(profix):
    sent_label = []
    with open(profix + '_sentence_transformer.pkl', 'rb') as f:
        data = pickle.load(f)
    for i in range(0, len(data)):
        labels = remodel.decode(np.array(data[i]))[1]
        sent_label.append(labels)
    
    with open(profix + '_sent_trans_cons_label.pkl', 'wb') as f:
        pickle.dump(sent_label, f)
    return sent_label

In [26]:
l = encode_convs('train')

In [27]:
l = encode_convs('val')

In [28]:
l = encode_convs('test')